In [1]:
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import reset_seeds


reset_seeds(seed_value=39)
dataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelIDs = ["linearSGD","svr","mlp"]
params = {}

data = featureExtraction(dataID,statorFreqs=[37],testsID=[21])  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[27],scaler_params=data.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
MODELS = []

for modelID in modelIDs:
    model = modelGenerator(modelID=modelID, data=data,params=params)
    model.train()
    MODELS.append(model)

2023-07-18 13:06:06.123974: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


training <class 'models.linearSGD.linearSGD'>
Test MSE: 4.901320469027718    Test MAE: 1.8583893799847278  
Test MSE: 1.4039918118314543    Test MAE: 0.9570821180950091  


/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 6)]               0         
                                                                 
 dense_0 (Dense)             (None, 130)               910       
                                                                 
 dense_1 (Dense)             (None, 140)               18340     
                                                                 
 dense (Dense)               (None, 1)                 141       
                                                                 
Total params: 19,391
Trainable params: 19,391
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300


2023-07-18 13:06:15.754305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 13:06:16.021664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


124/175 [====================>.........] - ETA: 0s - loss: 14708.1465 - mean_squared_error: 14708.1465 - mean_absolute_error: 117.7460

2023-07-18 13:06:16.659403: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


175/175 [==============================] - 1s 1ms/step - loss: 13622.6875 - mean_squared_error: 13622.6875 - mean_absolute_error: 112.7915 - val_loss: 9543.0479 - val_mean_squared_error: 9543.0479 - val_mean_absolute_error: 93.6938
Epoch 2/300
175/175 [==============================] - 0s 972us/step - loss: 7194.4712 - mean_squared_error: 7194.4712 - mean_absolute_error: 79.9705 - val_loss: 5026.0435 - val_mean_squared_error: 5026.0435 - val_mean_absolute_error: 66.0147
Epoch 3/300
175/175 [==============================] - 0s 962us/step - loss: 3789.1746 - mean_squared_error: 3789.1746 - mean_absolute_error: 56.7188 - val_loss: 2625.0796 - val_mean_squared_error: 2625.0796 - val_mean_absolute_error: 46.5579
Epoch 4/300
175/175 [==============================] - 0s 968us/step - loss: 2039.2649 - mean_squared_error: 2039.2649 - mean_absolute_error: 41.0430 - val_loss: 1462.6582 - val_mean_squared_error: 1462.6582 - val_mean_absolute_error: 34.6811
Epoch 5/300
175/175 [==================

In [2]:

X_test = dataTest.X
y_test = dataTest.y
# Selecting temps higher than 100ºC
idx = np.argwhere(y_test>100)[:,0]
y_test = y_test[idx]
X_test = X_test[idx,...]

Y_test_est = []
for model in MODELS:
    Y_test_est.append(model.predict(X_test))

8/8 [==============================] - 0s 603us/step


In [3]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()

MODEL_LABELS = {"mlp": "Multi layer perceptron","svr": "Support Vector Regression", "linearSGD":"Linear regression"}

for ii,model in enumerate(MODELS):
    p = figure(title=f"Residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=350)
    p.line(np.arange(len(y_test)),y_test, line_width=2,line_alpha=0.4, line_color='gray', legend_label="Ground truth" )
    p.line(np.arange(len(y_test)),Y_test_est[ii], line_width=2, line_alpha=0.8,line_color='green',legend_label="Estimation")
    p.legend.location = 'bottom_right'
    p.yaxis.axis_label = r"T (ºC)"
    p.xaxis.axis_label = r"sample"
    show(p)
    

Loading BokehJS ...

In [4]:
from scipy import signal

b = signal.firwin(151,0.01)
RESIDUALS = []
for ii,model in enumerate(MODELS):
    residuals = np.square(y_test-Y_test_est[ii])
    residuals_ = signal.filtfilt(b, 1, residuals)
    p = figure(title=f"Abs residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=250,y_range=(0,5))
    p.line(np.arange(len(y_test)),residuals, line_width=2, line_alpha=0.3,line_color='gray',legend_label="normal")
    p.line(np.arange(len(y_test)),residuals_, line_width=2, line_alpha=0.8,line_color='blue',legend_label="filtered")
    p.legend.location = 'top_right'
    p.yaxis.axis_label = r"$$|T-\hat{T}|^2_2$$"
    p.xaxis.axis_label = r"sample"
    show(p)
    RESIDUALS.append(residuals_)


## Boxplot residuals
Descriptive statisdics of the residuals obtained from applying all the models to all the avilable tests (#21, #24 and #27).  

In [38]:
#Boxplox residuals
from bokeh.models import ColumnDataSource, Whisker
from itertools import product

testIDs = [21,24,27]
TESTDATA = []
for test in testIDs: 
    dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[test],scaler_params=data.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
    X_test = dataTest.X
    y_test = dataTest.y
    # Selecting temps higher than 100ºC
    idx = np.argwhere(y_test>140)[:,0]
    y_test = y_test[idx]
    X_test = X_test[idx,...]
    TESTDATA.append((X_test,y_test))

# Get the residuals from all the tests

In [54]:
b = signal.firwin(151,0.01)
RESIDUALS = []
maxRes = {}
for test,model in product(testIDs[::-1],modelIDs):
    print(f"Test: {test} and model {model}  ")
    modelObj = MODELS[modelIDs.index(model)]
    X_test,y_test   = TESTDATA[testIDs.index(test)]
    y_test_est      = modelObj.predict(X_test)
    residuals = np.square(y_test-y_test_est)
    residuals_ = signal.filtfilt(b, 1, residuals)
    if test ==27:
        maxRes[model] = residuals_.max()
    

    residuals_ = residuals_/maxRes[model]
    df_residuals = pd.DataFrame(residuals_,columns=['Residuals'])
    df_residuals["Test"] = test
    df_residuals["Model"] = model
    RESIDUALS.append(df_residuals)

RESIDUALS = pd.concat(RESIDUALS,axis=0)

    

Test: 27 and model linearSGD  
Test: 27 and model svr  
Test: 27 and model mlp  
6/6 [==============================] - 0s 664us/step
Test: 24 and model linearSGD  
Test: 24 and model svr  
Test: 24 and model mlp  
6/6 [==============================] - 0s 711us/step
Test: 21 and model linearSGD  
Test: 21 and model svr  
Test: 21 and model mlp  
4/4 [==============================] - 0s 629us/step


In [124]:
from bokeh.models import ColumnDataSource, Whisker
#Extracting quartiles
Q_df = RESIDUALS.groupby(["Test","Model"]).quantile([0.25,0.5,0.75])
Q_df = Q_df.unstack().reset_index()
Q_df.columns = ["Test","Model","q1","q2","q3"]

# compute IQR 
iqr = Q_df.q3 - Q_df.q1
Q_df["upper"] = Q_df.q3 + 1.5*iqr
Q_df["lower"] = Q_df.q1 - 1.5*iqr

#Figure params 
colors = ["#BB8FCE", "#82E0AA","#F0B27A"]

p = figure( y_range=[-0.5,1.5],tools="", toolbar_location=None,
           title="Residuals per test",
           y_axis_label="residuals values")
for i,test in enumerate(testIDs):
    df = Q_df[Q_df.Test == test].copy()
    df.index = df.Model; df.index.name=None; df = df.loc[modelIDs]
    df["ModelLabel"] = pd.Categorical(df["Model"],categories=modelIDs).codes*4
    df.ModelLabel = df.ModelLabel + 0.7*i
    #df = df.reindex(modelIDs)
    source = ColumnDataSource(df)
    whisker = Whisker(base="ModelLabel", upper="upper", lower="lower", line_color="#626567", source=source)
    p.add_layout(whisker)
    p.vbar("ModelLabel", 0.6, "q2", "q3", source=source, legend_label = f"#{test}",  line_color="#626567", color=colors[i])
    p.vbar("ModelLabel", 0.6, "q1", "q2", source=source,  line_color="#626567", color=colors[i])

p.xaxis.ticker = [1, 5, 9]
p.xaxis.major_label_overrides = {1: 'MLR', 5: 'SVR', 9: 'MLP'}

show(p)

In [106]:
df.index.values

array(['linearSGD', 'svr', 'mlp'], dtype=object)

In [83]:
Q_df.ModelLabel = pd.Categorical(Q_df["Model"]).codes*4
Q_df.ModelLabel

array([0, 4, 8, 0, 4, 8, 0, 4, 8], dtype=int8)

In [117]:

df

,Test,Model,q1,q2,q3,upper,lower,ModelLabel
linearSGD,27,linearSGD,0.808576,0.825157,0.915577,1.076079,0.648074,1.4
svr,27,svr,0.520994,0.687288,0.785070,1.181185,0.124880,5.4
mlp,27,mlp,0.099738,0.128681,0.334929,0.687717,-0.253049,9.4
